<a href="https://colab.research.google.com/github/ayyucedemirbas/machine_learning_algorithms/blob/master/lpips_my_repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ayyucedemirbas/lpips-tensorflow.git

Cloning into 'lpips-tensorflow'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 73 (delta 1), reused 0 (delta 0), pack-reused 67
Unpacking objects: 100% (73/73), done.


In [2]:
%cd lpips-tensorflow

/content/lpips-tensorflow


In [3]:
%%writefile test_network.py
import argparse

import cv2
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import lpips_tf


def load_image(fname):
    image = cv2.imread(fname)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image.astype(np.float32) / 255.0


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', choices=['net-lin', 'net'], default='net-lin', help='net-lin or net')
    parser.add_argument('--net', choices=['squeeze', 'alex', 'vgg'], default='vgg', help='squeeze, alex, or vgg')
    parser.add_argument('--version', type=str, default='0.1')
    args = parser.parse_args()

    ex_ref = load_image('2.png')
    ex_p0 = load_image('1.png')
    ex_p1 = load_image('0.png')

    session = tf.Session()

    image0_ph = tf.placeholder(tf.float32)
    image1_ph = tf.placeholder(tf.float32)
    lpips_fn = session.make_callable(
        lpips_tf.lpips(image0_ph, image1_ph, model=args.model, net=args.net, version=args.version),
        [image0_ph, image1_ph])

    ex_d0 = lpips_fn(ex_ref, ex_p0)
    ex_d1 = lpips_fn(ex_ref, ex_p1)

    print('Distances: (%.3f, %.3f)' % (ex_d0, ex_d1))


if __name__ == '__main__':
    main()

Overwriting test_network.py


In [4]:
!python test_network.py

Instructions for updating:
non-resource variables are not supported in the long term
2021-12-13 22:05:12.756018: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
>> Downloading net-lin_vgg_v0.1.pb 100.0%
Successfully downloaded net-lin_vgg_v0.1.pb 58955824 bytes.
2021-12-13 22:05:36.099135: W tensorflow/core/common_runtime/graph_constructor.cc:1511] Importing a graph with a lower producer version 27 into an existing graph with producer version 898. Shape inference will have run different parts of the graph with different producer versions.
Distances: (0.593, 0.565)
